In [1]:
import tensorflow as tf
import numpy as np
import math
print(tf.__version__)

1.12.0


In [2]:
import mnist_reader
train_images, train_labels = mnist_reader.load_mnist('data', kind='train')
test_images, test_labels = mnist_reader.load_mnist('data', kind='t10k')
train_num = len(train_labels)
test_num = len(test_labels)

In [3]:
train_images = train_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)

In [4]:
train_labels_onehot = np.ones((len(train_labels), 10)) * 0
for i in range(len(train_labels)):
    train_labels_onehot[i][train_labels[i]] = 1

In [5]:
test_labels_onehot = np.ones((len(test_labels), 10)) * 0
for i in range(len(test_labels)):
    test_labels_onehot[i][test_labels[i]] = 1

In [6]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [7]:
def conv2d(x, w):
    return tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [8]:
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, 10])

w_conv1 = tf.Variable(tf.truncated_normal([3, 3, 1, 32], stddev=0.1))
b_conv1 = tf.Variable(tf.constant(0.1, shape=[32]))
h_conv1 = tf.nn.relu(conv2d(x, w_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

w_conv2=tf.Variable(tf.truncated_normal([3, 3, 32, 50], stddev=0.1))
b_conv2=tf.Variable(tf.constant(0.1, shape=[50]))
h_conv2=tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2)
h_pool2=max_pool_2x2(h_conv2)

w_fc1 = tf.Variable(tf.truncated_normal([7*7*50, 1024], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*50])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

w_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1))
b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))
y_out = tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2) + b_fc2)

loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_out), reduction_indices=[1]))
#cross_entropy = -tf.reduce_sum(y * tf.log(y_out))
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
#train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_out, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess=tf.InteractiveSession()
#sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [9]:
batch_size = 50
batch_num = math.floor(train_num / batch_size)
    
for i in range(4000):
    if (i % 1000 == 0):
        index = i % batch_num * batch_size
        train_accuracy = accuracy.eval(session=sess, feed_dict={x:train_images[index:index+batch_size], y:train_labels_onehot[index:index+batch_size], keep_prob:1.0})
        print("step: ", i, "train_accuracy: ", train_accuracy)
    index = i % batch_num * batch_size
    train_step.run(session=sess, feed_dict={x:train_images[index:index+batch_size], y:train_labels_onehot[index:index+batch_size], keep_prob:0.5})

step:  0 train_accuracy:  0.06
step:  1000 train_accuracy:  0.06
step:  2000 train_accuracy:  0.16
step:  3000 train_accuracy:  0.08
